# Transformations and the Scheduler

The purpose of this notebook is to explain how Loki `Transformation`s are defined and used, and explain how transformations can be used to process entire call trees using the `Scheduler`. This notebook assumes that the reader is already familiar with the basics of Loki.

## Defining a transformation as a Python class

Recall that to implement a new transformation as a Python class in Loki, one must define a class that:
1. Inherits from the abstract class `loki.Transformation`.
2. Implements _at least one_ of the methods making up the so-called "transformation interface" of Loki:
    - `transform_subroutine(routine: loki.Subroutine, **kwargs) -> None`,
    - `transform_module(module: loki.Module, **kwargs) -> None`,
    - `transform_file(sourcefile: loki.Sourcefile, **kwargs) -> None`.
    
The methods making up the transformation interface describe how the transformation should process subroutines, modules and whole sourcefiles, respectively. Note that the user is free to pass arbitrary data (needed in the transformation) to the transformation in its constructor. 

As a simple example, consider a transformation that removes a dummy argument with a certain name from `Subroutine`s.

In [7]:
from loki import Transformation, FindNodes, VariableDeclaration, Transformer
class RemoveDummyArgTransformation(Transformation):
    # NOTE: here the constructor of the transformation takes the string `to_remove` as input.
    def __init__(self, to_remove):
        super().__init__()
        self.to_remove = to_remove

    def transform_subroutine(self, routine, **kwargs):
        # Remove dummy argument from argument list.
        routine.arguments = (var for var in routine.arguments if var.name != self.to_remove)

        # Remove dummy argument from specification.
        transmap = {}
        for vd in FindNodes(VariableDeclaration).visit(routine.spec):
            if self.to_remove in (v.name for v in vd.symbols):
                new_symbols = [var.clone() for var in vd.symbols if var.name != self.to_remove]
                if new_symbols:
                    transmap[vd] = vd.clone(symbols = new_symbols)
                else:
                    transmap[vd] = None
        routine.spec = Transformer(transmap).visit(routine.spec)

    # Behaviour in case of a Sourcefile is to apply the transformation 
    # to all subroutines in the file, and print the name of the routine just before transforming it.
    def transform_file(self, file, **kwargs):
        for routine in file.all_subroutines:
            print(f"Processing routine: {routine.name}")
            self.transform_subroutine(routine, **kwargs)

We may apply the transformation to some example code as follows:

In [8]:
from loki import Sourcefile
fcode = """
SUBROUTINE test(x, y)
    INTEGER, INTENT(IN) :: x
    INTEGER, INTENT(INOUT) :: y
    y = 4 + x
END SUBROUTINE test
"""
src = Sourcefile.from_source(fcode)

# Create transformation to remove a dummy argument named `x`.
transformation = RemoveDummyArgTransformation(to_remove = "x")

# Apply the transformation.
transformation.apply(src)

# Check results after transforming.
print(src.to_fortran())

Processing routine: test

SUBROUTINE test (y)
  INTEGER, INTENT(INOUT) :: y
  y = 4 + x
END SUBROUTINE test


Note that Loki comes with a number of predefined transformations: https://sites.ecmwf.int/docs/loki/main/transform.html 

## Processing source trees using the `Scheduler`

The above example showed how to make a transformation at the level of an individual subroutine, module or sourcefile. However, in practice, most source code bases have multiple sourcefiles that can have complex interdependencies. In such cases, the approach of modifying single entities at a time may become inefficient. Next, we will look at how we can use Loki to more effectively work in such situations, and apply transformations or even transformation pipelines (consisting of multiple transformations in sequence) at the level of full "codebases". The main tool to facilitate this is the `Scheduler` object.

As an example, we will look at a "codebase" consisting of three subroutines 'wrapper', 'depth1' and 'depth2' in three separate files. The subroutines form the following call tree:
- 'wrapper' in 'wrapper.F90' calls 'depth1' in 'depth1.F90'
- depth1 calls 'depth2' in 'depth2.F90'

**Subroutine 'wrapper'**

In [20]:
print(open("src/wrapper.F90", "r").read())

subroutine wrapper
    implicit none

    integer :: h_start = 1
    integer, parameter :: h_dim = 10
    integer :: h_end 
    integer :: v_start = 1
    integer, parameter :: v_dim = 20
    integer, parameter :: block_size = 30
    integer :: block_index

    real :: arr(h_dim, v_dim, block_dim)

    h_end = h_dim
    do block_index = 1, block_size 
        call depth1(h_start, h_end, h_dim, v_start, v_dim, &
                    arr(:, :, block_index))
    end do
    print *, "Sum of array is ", sum(arr)
end subroutine wrapper



**Subroutine 'depth1'**

In [21]:
print(open("src/depth1.F90", "r").read())

subroutine depth1(h_start, h_end, h_dim, v_start, v_dim, array)
    implicit none

    ! Arguments.
    integer, intent(in) :: h_start
    integer, intent(in) :: h_end
    integer, intent(in) :: h_dim
    integer, intent(in) :: v_start
    integer, intent(in) :: v_dim
    real, intent(inout) :: array(h_dim, v_dim)

    ! Non-array local variables.
    integer :: v_index
    integer :: h_index

    ! Temporary arrays.
    real :: tmp1(h_dim, v_dim)

    do v_index = v_start, v_dim
        do h_index = h_start, h_end
            tmp1(h_index, v_index) = exp(log(real(h_index)) + log(real(v_index)) - 1.0)
        end do
    end do

    do v_index = v_start, v_dim
        do h_index = h_start, h_end
            array(h_index, v_index) = exp(tmp1(h_index, v_index) + 0.25)
        end do
    end do

    call depth2(h_start, h_end, h_dim, v_start, v_dim, array)

    do v_index = v_start, v_dim
        do h_index = h_start, h_end
            array(h_index, v_index) = log(tmp1(h_index, v_index))

**Subroutine 'depth2'**

In [22]:
print(open("src/depth2.F90", "r").read())

subroutine depth2(h_start, h_end, h_dim, v_start, v_dim, array)
    implicit none
    integer, intent(in) :: h_start
    integer, intent(in) :: h_end
    integer, intent(in) :: h_dim
    integer, intent(in) :: v_start
    integer, intent(in) :: v_dim
    real, intent(inout) :: array(h_dim, v_dim)

    ! Non-array local variables.
    integer :: v_index
    integer :: h_index
    real :: val

    ! Temporary arrays.
    real :: tmp2(h_dim, v_dim)

    val = 1.0
    call contained(val)
    tmp2 = 2.0 + val
    tmp2(h_dim, v_dim) = -1.0

    do v_index = v_start, v_dim
        do h_index = h_start, h_end
            array(h_index, v_index) = array(h_index, v_index) + tmp2(h_index, v_index)
        end do
    end do
contains

    subroutine contained(x)
        real, intent(inout) :: x
        x = x * 2.0
    end subroutine contained

end subroutine depth2



### Applying a transformation to a full code base

NOTE, TODO: I think somewhere in this section it would make sense to explicitly list what data the Scheduler is passing to the transformations. Perhaps there are other ways of passing data also, that have not been covered here.

To make transformations for the full code base, we need to first construct a `Scheduler` object, which in summary is responsible for:

1. Building a call tree of the program to be processed
2. Managing the application of transformations to the call tree

Constructing a `Scheduler` requires some configuration in the form of a `SchedulerConfig` object. The purpose of `SchedulerConfig` is to store data on how individual subroutines (and modules and sourcefiles?) should be transformed (TODO: Maybe something else as well?). As we will see in a bit, this data is passed by the `Scheduler` to the transformations that are eventually applied to transform the full program.

To make this more concrete, let's consider an example where we would like to rename (some of) the subroutines in the call tree formed by the Fortran subroutines above. We write the following transformation (intended to be used with the `Scheduler`):

In [58]:
class RenameWithSchedulerTransformation(Transformation):
    def __init__(self):
        super().__init__()

    def transform_subroutine(self, routine, **kwargs):
        # The Scheduler implicitly passes a field named 'item' to the keyword arguments of the transformation.
        item = kwargs.get('item', None)
        newname = None
        if item:
            newname = item.config['newname'] 
        if newname:
            print(f"Found a new name '{newname}' when processing subroutine '{routine.name}', renaming..")
            routine.name = newname

Note that in contrast to the `RemoveDummyArgTransformation` in the previous section, the above transformation takes no input data in its constructor. Rather, the transformation relies on the workings of the `Scheduler`, which passes the data (in this case, a variable named `newname`) to the transformation via a keyword argument `item`. The data can be accessed from the dictionary `item.config`, which (inside the transformation) corresponds to the relevant part of a `SchedulerConfig` object.

Passing data to transformations using this mechanism can be thought of as an alternative way of passing data to transformations (as opposed to passing the data to the constructor of the transformation as in the previous section).

To actually define what `item.config` above should (eventually) contain, we need to construct the `SchedulerConfig`:

In [59]:
from loki import SchedulerConfig
# The configuration for subroutine 'depth2' and 'wrapper':
routine_specific_config = {
    'depth2': {'newname': 'depth2_modified'}, # For 'depth2', 'newname' is 'depth2_modified'
    'wrapper': {'newname': 'i_am_the_wrapper', 'some_other_data': [1, 2, 3]} # For 'wrapper2', 'newname' is 'i_am_the_wrapper'.
}
# The 'base configuration' (for every other subroutine, and overrided by `routine_specific_config`):
default_config = {
    'expand': True,
    'newname': None # By default, 'newname' is None (i.e no change is made to name)
}
# Construct SchedulerConfig.
sconfig = SchedulerConfig(routines = routine_specific_config, default = default_config)

Note that in the above configuration, we have configured the wrapper routine to also receive `some_other_data` with a value of `[1, 2, 3]`, which is not used by `RenameWithSchedulerTransformation` (but could be used by some other transformation). Furthermore, the default configuration contains `expand = True`, which tells the Scheduler to (what exactly???) (TODO: I think this is going to be removed in a further version) Note that Loki also provides a way to read the scheduler configuration from a file via `SchedulerConfig.from_file`.

With the configuration defined, we may then proceed to construct a `Scheduler` object. Here, we pass the arguments:

* `paths`: folder paths where the Fortran files should be searched for.
* `config`: The scheduler configuration.

In [60]:
from loki import Scheduler
scheduler = Scheduler(paths = "src", config = sconfig)

[Loki::Scheduler] Performed initial source scan in 0.02s
[Loki::Sourcefile] Finished constructing from src/depth2.F90 in 0.03s
[Loki::Sourcefile] Finished constructing from src/depth1.F90 in 0.05s
[Loki::Sourcefile] Finished constructing from src/wrapper.F90 in 0.02s
[Loki::Scheduler] Performed full source parse in 0.10s


From the message above, we see that the `Scheduler` was able to locate all dependencies of 'wrapper'. Indeed, we can inspect the `items` that the `Scheduler` found:

In [61]:
scheduler.items

(loki.bulk.Item<#depth2>, loki.bulk.Item<#wrapper>, loki.bulk.Item<#depth1>)

Each item represents a work item to be processed by the `Scheduler`. We are now ready to "bulk apply" our transformation to the full call tree, with the following call: 

In [62]:
scheduler.process(transformation = RenameWithSchedulerTransformation())

[Loki::Scheduler] Applied transformation <RenameWithSchedulerTransformation> in 0.00s


Found a new name 'i_am_the_wrapper' when processing subroutine 'wrapper', renaming..
Found a new name 'depth2_modified' when processing subroutine 'depth2', renaming..


If we now inspect the names of the routines from inside the `Scheduler` object, we see that the changes were indeed made:

In [63]:
[item.routine.name for item in scheduler.items]

['depth2_modified', 'i_am_the_wrapper', 'depth1']

Note that had we defined further transformations, we could process them in sequence by repeatedly calling `scheduler.process` for each transformation, effectively creating a "transformation pipeline". Finally, note that it is possibly to mix different ways of passing data to transformations, depending on the use case. It is for example possible to pass some of the transformation data via `SchedulerConfig`, and the rest via the constructors of transformations.